In [6]:
import torch
import sys
sys.path.append("..")

import json, time, os
from statsmodels.tsa.arima.model import ARIMA
from utils import get_gnn_data
from log.logutli import Logger

In [2]:
x = torch.randn(64, 134, 288, 10)

print(x.shape)

torch.Size([64, 134, 288, 10])


In [22]:
with open('../config.json', 'r') as f:
    config = json.load(f)
config['data_path'] = '../data/'
config["location_path"] = "../data/sdwpf_baidukddcup2022_turb_location.CSV"


# "data_path"      : "./data/",
#     "filename"       : "wtbdata_245days.csv",
#     "location_path"  : "./data/sdwpf_baidukddcup2022_turb_location.CSV",

start_time   = time.time()
current_time = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime(start_time))
save_dir     = os.path.join("../result", current_time + f'_test')
log_id       = 'main'
log_name     = f'Run_{current_time}.log'
log_level    = 'info'
Logger_      = Logger(log_id, save_dir, log_name, log_level)
logger       = Logger_.logger
logger.info(f"LOCAL TIME: {current_time}")

train_original_data, val_original_data, test_original_data, _, _, _ = get_gnn_data(config, logger)  

2024-01-08 17:32:09-main-INFO: LOCAL TIME: 2024_01_08_17_32_09


log_name: Run_2024_01_08_17_32_09.log


2024-01-08 17:32:14-main-INFO: Raw data.shape: (4727520, 13)
2024-01-08 17:32:15-main-INFO: data.shape: (134, 10, 35280), means.shape: (10,), stds.shape: (10,)
2024-01-08 17:32:15-main-INFO: train_original_data: (134, 10, 28800), 
val_original_data: (134, 10, 2880), 
test_original_data: (134, 10, 3600).


In [11]:
x1 = train_original_data[0, -1, :]
print(f'x1.shape: {x1.shape}')
input_time_step = 48
forecast_steps = 24

train, test = x1[:input_time_step], x1[input_time_step:input_time_step+forecast_steps]

# train_size = 80
# train, test = data[:train_size], data[train_size:]

# 训练ARIMA模型
order = (2, 1, 2)  # ARIMA(p, d, q)参数，可以根据实际情况调整
model = ARIMA(train, order=order)
model_fit = model.fit()

# 多步向前预测
forecast = model_fit.forecast(steps=forecast_steps)
print(forecast.shape)

x1.shape: (28800,)
(24,)


In [12]:
print(test)
print(forecast)

[2.79396946 2.79439029 2.75946172 2.69734311 2.71969366 2.79457732
 2.7460888  2.79417988 2.78728301 2.69851208 2.77968476 2.70279047
 2.7347265  2.72114317 2.5437182  2.19223488 2.03687991 1.43818463
 1.6110974  1.38177055 1.28972653 1.00132037 0.63980736 0.94687015]
[2.7936841  2.79360479 2.79356603 2.79354963 2.79354252 2.79353946
 2.79353814 2.79353757 2.79353732 2.79353722 2.79353717 2.79353715
 2.79353714 2.79353714 2.79353714 2.79353714 2.79353714 2.79353714
 2.79353714 2.79353714 2.79353714 2.79353714 2.79353714 2.79353714]


In [23]:
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

train_indices = [(i, i + (config['input_len'] + config['output_len'])) 
        for i in range(train_original_data.shape[2] - \
                        (config['input_len'] + config['output_len']) + 1)]
val_indices = [(i, i + (config['input_len'] + config['output_len'])) 
        for i in range(val_original_data.shape[2] - \
                        (config['input_len'] + config['output_len']) + 1)]

print(test_original_data.shape)


for turb_id in range(test_original_data.shape[0]):  # [num_nodes, num_features, seq_len]
    data = test_original_data[turb_id, -1, :]  # [seq_len, ]
    print(data.shape)
    for i in tqdm(range(test_original_data.shape[2] - \
                        (config['input_len'] + config['output_len']) + 1)):
        train, test = data[i:i+input_time_step], data[i+input_time_step:i+input_time_step+forecast_steps]

        # 训练ARIMA模型
        order = (1, 1, 1)  # ARIMA(p, d, q)参数，可以根据实际情况调整
        model = ARIMA(train, order=order)
        model_fit = model.fit()

        # 多步向前预测
        forecast = model_fit.forecast(steps=forecast_steps)
        # print(forecast.shape)
    if turb_id > 1:
        break 

(134, 10, 3600)
(3600,)


  0%|          | 0/3529 [00:00<?, ?it/s]

100%|██████████| 3529/3529 [02:51<00:00, 20.59it/s]


(3600,)


100%|██████████| 3529/3529 [02:58<00:00, 19.82it/s]


(3600,)


100%|██████████| 3529/3529 [03:00<00:00, 19.55it/s]


In [29]:
X = test_original_data[66, -1, :]
for i in tqdm(range(len(X) - \
                    (config['input_len'] + config['output_len']) + 1)):
    train, test = X[i:i+input_time_step], X[i+input_time_step:i+input_time_step+forecast_steps]

    order = (1, 1, 1)  # ARIMA(p, d, q)参数，可以根据实际情况调整
    model = ARIMA(X, order=order)
    model_fit = model.fit()

    # 多步向前预测
    forecast = model_fit.forecast(steps=forecast_steps)

100%|██████████| 3529/3529 [41:11<00:00,  1.43it/s]


In [27]:
X.shape

(3600,)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
batch_size       = 64
num_of_timesteps = 48
num_of_nodes     = 132
num_of_features  = 10
num_filters      = 64
num_time_filter  = 64
time_strides     = 1

x = torch.randn(batch_size, num_of_nodes, num_of_features, num_of_timesteps)
print(f'x.shape: {x.shape}')

x2 = torch.randn(batch_size, num_of_nodes, num_filters, num_of_timesteps) 
print(f'x2.shape: {x2.shape}')


# temporal_At = torch.randn(batch_size, num_of_timesteps, num_of_timesteps)
time_conv = nn.Conv2d(num_filters, num_time_filter, kernel_size=(1, 3), stride=(1, time_strides), padding=(0, 1))
x3 = time_conv(x2.permute(0, 2, 1, 3))  # [bs, n, f, t] -> [bs, f, n, t]
print(f'x3.shape: {x3.shape}')

x.shape: torch.Size([64, 132, 10, 48])
x2.shape: torch.Size([64, 132, 64, 48])
x3.shape: torch.Size([64, 64, 132, 48])


In [5]:
from torch.nn.utils import weight_norm 

class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        """
        其实这就是一个裁剪的模块，裁剪多出来的padding
        """
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        """
        相当于一个Residual block

        :param n_inputs: int, 输入通道数
        :param n_outputs: int, 输出通道数
        :param kernel_size: int, 卷积核尺寸
        :param stride: int, 步长，一般为1
        :param dilation: int, 膨胀系数
        :param padding: int, 填充系数
        :param dropout: float, dropout比率
        """
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        # 经过conv1，输出的size其实是(Batch, input_channel, seq_len + padding)
        self.chomp1   = Chomp1d(padding)  # 裁剪掉多出来的padding部分，维持输出时间步为seq_len
        self.relu1    = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2   = Chomp1d(padding)  # 裁剪掉多出来的padding部分，维持输出时间步为seq_len
        self.relu2    = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(
            n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        """
        参数初始化

        :return:
        """
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        """
        :param x: size of (Batch, input_channel, seq_len)
        :return:
        """
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


In [27]:
kernel_size = 3
stride = 1
dilation = 2

xx = torch.randn(batch_size, num_of_features, num_of_timesteps)
print(f'xx.shape: {xx.shape}')

tcn_block = TemporalBlock(num_of_features, num_of_timesteps, kernel_size, stride, dilation, padding=(kernel_size - 1)*dilation)
xx = tcn_block(xx)
print(f'xx.shape: {xx.shape}')

xx.shape: torch.Size([64, 10, 48])
xx.shape: torch.Size([64, 48, 48])


In [25]:
layers = []
num_channels = [25, 25, 25, 25]
num_layers = len(num_channels)
for i in range(num_layers):
    dilation_size = 2 ** i
    in_channels = num_of_features if i == 0 else num_channels[i - 1]
    out_channels = num_channels[i]
    print(f'i: {i}, dilation: {dilation_size}, in_channels: {in_channels}, out_channels: {out_channels}')
    layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride, dilation_size, padding=(kernel_size - 1)*dilation_size)]

net = nn.Sequential(*layers)
net

i: 0, dilation: 1, in_channels: 10, out_channels: 25
i: 1, dilation: 2, in_channels: 25, out_channels: 25
i: 2, dilation: 4, in_channels: 25, out_channels: 25
i: 3, dilation: 8, in_channels: 25, out_channels: 25


Sequential(
  (0): TemporalBlock(
    (conv1): Conv1d(10, 25, kernel_size=(3,), stride=(1,), padding=(2,))
    (chomp1): Chomp1d()
    (relu1): ReLU()
    (dropout1): Dropout(p=0.2, inplace=False)
    (conv2): Conv1d(25, 25, kernel_size=(3,), stride=(1,), padding=(2,))
    (chomp2): Chomp1d()
    (relu2): ReLU()
    (dropout2): Dropout(p=0.2, inplace=False)
    (net): Sequential(
      (0): Conv1d(10, 25, kernel_size=(3,), stride=(1,), padding=(2,))
      (1): Chomp1d()
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
      (4): Conv1d(25, 25, kernel_size=(3,), stride=(1,), padding=(2,))
      (5): Chomp1d()
      (6): ReLU()
      (7): Dropout(p=0.2, inplace=False)
    )
    (downsample): Conv1d(10, 25, kernel_size=(1,), stride=(1,))
    (relu): ReLU()
  )
  (1): TemporalBlock(
    (conv1): Conv1d(25, 25, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(2,))
    (chomp1): Chomp1d()
    (relu1): ReLU()
    (dropout1): Dropout(p=0.2, inplace=False)
    (conv2): Conv1d(25

In [26]:
xx = torch.randn(batch_size, num_of_features, num_of_timesteps)
print(f'xx.shape: {xx.shape}')

xx2 = net(xx)  
print(f'xx2.shape: {xx2.shape}')

xx.shape: torch.Size([64, 10, 48])
xx2.shape: torch.Size([64, 25, 48])


In [28]:
pad = kernel_size - 1
chomp = Chomp1d(pad)
relu = nn.ReLU()

a1 = torch.randn(batch_size, num_of_features, num_of_timesteps)
print(f'a1.shape: {a1.shape}')

d1 = 1
pad1 = (kernel_size-1)*d1
chomp1 = Chomp1d(pad1)
conv1 = nn.Conv1d(num_of_features, 25, kernel_size, stride=stride, padding=pad1, dilation=d1)
a2 = conv1(a1)
print(f'before chomp: a2.shape: {a2.shape}')
a2 = chomp1(a2)
a2 = relu(a2)
print(f'a2.shape: {a2.shape}')

d2 = 2
pad2 = (kernel_size-1)*d2
chomp2 = Chomp1d(pad2)
conv2 = nn.Conv1d(25, 25, kernel_size, stride=stride, padding=(kernel_size-1)*d2, dilation=d2)
a3 = conv2(a2)
a3 = chomp2(a3)
a3 = relu(a3)
print(f'a3.shape: {a3.shape}')


a1.shape: torch.Size([64, 10, 48])
before chomp: a2.shape: torch.Size([64, 25, 50])
a2.shape: torch.Size([64, 25, 48])
a3.shape: torch.Size([64, 25, 48])


## 3维TCN

In [31]:
class Chomp2d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp2d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        """
        裁剪多余的padding。

        Args:
            x: [batch_size, num_nodes, num_features, num_time_steps]
        """
        return x[:, :, :, : -self.chomp_size].contiguous()
    

In [35]:
batch_size       = 64
num_of_timesteps = 48
num_of_nodes     = 132
num_of_features  = 10
num_filters      = 64
num_time_filter  = 64
time_strides     = 1
kernel_size      = 3

x = torch.randn(batch_size, num_of_nodes, num_of_features, num_of_timesteps)
print(f'x.shape: {x.shape}')

# x2 = torch.randn(batch_size, num_of_nodes, num_filters, num_of_timesteps) 
# print(f'x2.shape: {x2.shape}')


# temporal_At = torch.randn(batch_size, num_of_timesteps, num_of_timesteps)
# time_conv = nn.Conv2d(num_filters, num_time_filter, kernel_size=(1, kernel_size), stride=1, padding=(0, 1))
# x3 = time_conv(x.permute(0, 2, 1, 3))  # [bs, n, f, t] -> [bs, f, n, t]
# print(f'x3.shape: {x3.shape}')

d1 = 1
pad1 = (kernel_size-1)*d1
chomp1 = Chomp2d(pad1)
# conv1 = nn.Conv2d(num_of_features, 25, kernel_size, stride=stride, padding=pad1, dilation=d1)
conv1 = nn.Conv2d(num_of_features, num_time_filter, kernel_size=(1, kernel_size), stride=1, dilation=d1, padding=(0, (kernel_size-1)*d1))
a2 = conv1(x.permute(0, 2, 1, 3)) # [bs, n, f, t] -> [bs, f, n, t] -> [bs, f, n, t]
a2 = chomp1(a2)
a2 = relu(a2)
print(f'a2.shape: {a2.shape}')

d2 = 1
pad2 = (kernel_size-1)*d2
chomp2 = Chomp2d(pad2)
conv2 = nn.Conv2d(num_time_filter, num_time_filter, kernel_size=(1, kernel_size), stride=1, padding=(0, (kernel_size-1)*d2), dilation=d2)
a3 = conv2(a2)
a3 = chomp2(a3)
a3 = relu(a3)
print(f'a3.shape: {a3.shape}')

x.shape: torch.Size([64, 132, 10, 48])
a2.shape: torch.Size([64, 64, 132, 48])
a3.shape: torch.Size([64, 64, 132, 48])


In [63]:
class TemporalBlock2d(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, dropout=0.2):
        super(TemporalBlock2d, self).__init__()
        padding  = (kernel_size - 1) * dilation
        self.conv1 = weight_norm(nn.Conv2d(n_inputs, n_outputs, kernel_size=(1, kernel_size),
                                           stride=stride, padding=(0, padding), dilation=dilation))
        self.chomp1   = Chomp2d(padding)
        self.relu1    = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv2d(n_outputs, n_outputs, kernel_size=(1, kernel_size),
                                           stride=stride, padding=(0, padding), dilation=dilation))
        self.chomp2   = Chomp2d(padding)
        self.relu2    = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv2d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)
        
    def forward(self, x):
        """
        Args:
            x: [batch_size, num_nodes, num_features, num_time_steps]
        """
        print(f'block: x.shape: {x.shape}')
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)
    
class TCN2d(nn.Module):
    def __init__(self, 
                 input_size: int = 10, 
                 output_size: int = 64,
                 num_channels: list = [64, 128, 256], 
                 kernel_size: int = 3, 
                 dropout: float = 0.2,
                 device: str = 'cpu'):
        super(TCN2d, self).__init__()
        self.device = device

        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = input_size if i == 0 else num_channels[i - 1]
            out_channels = num_channels[i]
            print(f'i: {i}, dilation: {dilation_size}, in_channels: {in_channels}, out_channels: {out_channels}')
            layers += [TemporalBlock2d(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)
        self.linear  = nn.Linear(num_channels[-1], output_size)

    def forward(self, x):
        """
        Args:
            x: [batch_size, num_nodes, num_features, num_time_steps]
        """

        x = x.permute(0, 2, 1, 3)  # x: [bs, n, f, t] -> [bs, f, n, t]
        print(f'tcn input: x.shape: {x.shape}')
        x = x.to(self.device)
        x = self.network(x)  # [bs, f, n, t] -> [bs, num_channels[-1], n, t]
        print(f'tcn network output: x.shape: {x.shape}')
        x = self.linear(x.permute(0, 2, 3, 1)) # [bs, num_channels[-1], n, t] -> [bs, n, t, num_channels[-1]] -> [bs, n, t, out_channels]
        print(f'tcn linear output: x.shape: {x.shape}')
        return x.permute(0, 3, 1, 2)  # [bs, n, t, out_channels] -> [bs, out_channels, n, t]

In [64]:
x = torch.randn(batch_size, num_of_nodes, num_of_features, num_of_timesteps)
print(f'x.shape: {x.shape}')

block1 = TemporalBlock2d(num_of_features, num_time_filter, kernel_size, stride, dilation=1)
x1 = block1(x.permute(0, 2, 1, 3)) # x: [bs, n, f, t] -> [bs, f, n, t] -> [bs, f, n, t]
print(f'x1.shape: {x1.shape}')

num_channels = [16, 32, 64]
kernel_size = 3
tcn = TCN2d(num_of_features, num_time_filter, num_channels, kernel_size)
x2 = tcn(x)
print(f'x2.shape: {x2.shape}')

x.shape: torch.Size([64, 132, 10, 48])
block: x.shape: torch.Size([64, 10, 132, 48])
x1.shape: torch.Size([64, 64, 132, 48])
i: 0, dilation: 1, in_channels: 10, out_channels: 16
i: 1, dilation: 2, in_channels: 16, out_channels: 32
i: 2, dilation: 4, in_channels: 32, out_channels: 64
tcn input: x.shape: torch.Size([64, 10, 132, 48])
block: x.shape: torch.Size([64, 10, 132, 48])
block: x.shape: torch.Size([64, 16, 132, 48])
block: x.shape: torch.Size([64, 32, 132, 48])
tcn network output: x.shape: torch.Size([64, 64, 132, 48])
tcn linear output: x.shape: torch.Size([64, 132, 48, 64])
x2.shape: torch.Size([64, 64, 132, 48])
